In [1]:
import dspy
import hashlib
from tqdm import tqdm
import chromadb
import weaviate
from dspy.retrieve.weaviate_rm import WeaviateRM
import tiktoken
encoding = tiktoken.get_encoding("cl100k_base")
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")
from dspy.teleprompt import BootstrapFewShot 

c:\Users\PedroLustosa\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import time
import json


In [3]:
import os
from dotenv import load_dotenv
import openai
from openai import OpenAI
from dotenv import load_dotenv

# Acessa a chave de API
secretk = os.getenv("OPENAI_API_KEY")

#client = OpenAI(api_key=secretk)
openai.api_key = secretk
client = OpenAI(api_key=secretk)


In [4]:
def create_few_shot_from_json(file_path, limit=10):
    """
    Carrega exemplos few-shot a partir de um arquivo JSON, com limite opcional.

    Args:
        file_path: O caminho para o arquivo JSON contendo os dados.
        limit: O número máximo de exemplos a serem retornados (padrão: 10).

    Returns:
        Uma lista de listas formatadas para few-shot learning, limitada ao número especificado.
    """
    try:
        with open(file_path, 'r') as file:
            data_dict = json.load(file)

        # Converter o dicionário em uma lista de listas e limitar o número de exemplos
        few_shot = [[key, ", ".join(map(str, value))] for key, value in data_dict.items()][:limit]
        
        return few_shot
    except FileNotFoundError:
        print(f"Arquivo não encontrado: {file_path}")
        return []
    except json.JSONDecodeError:
        print(f"Erro ao decodificar o JSON em: {file_path}")
        return []




In [5]:
# Uso da função
few_shot = create_few_shot_from_json('filtered_diseases_symptoms_Medlineplus.json', limit=10)
few_shot

[['HIV',
  'Fever, Chills, Rash, Night sweats (heavy sweating during sleep), Muscle aches, Sore throat, Fatigue, Swollen lymph nodes, Mouth ulcers'],
 ['Acute Flaccid Myelitis',
  'Arm or leg weakness, A loss of muscle tone and reflexes, Facial drooping/weakness, Trouble moving the eyes, Drooping eyelids, Trouble swallowing, Slurred speech, Pain in the arms, legs, back, or neck'],
 ['Bird Flu',
  'Fever (but not everyone has a fever), Cough, Sore throat, Runny or stuffy nose, Muscle or body aches, Fatigue, Headaches, Eye redness (conjunctivitis), Trouble breathing, Diarrhea'],
 ['Sepsis',
  'Rapid breathing and heart rate, Shortness of breath, Confusion or disorientation, Extreme pain or discomfort, Fever, shivering, or feeling very cold, Clammy or sweaty skin'],
 ['Dengue',
  'A high fever, Nausea and vomiting, Rash, Aches and pains (eye pain, usually behind the eyes, and pain in the muscles, joints, or bones), Severe abdominal (belly) pain and tenderness, Vomiting at least 3 times in

In [6]:
treino = [dspy.Example(question=few_shot[x][0], 
                       answer=few_shot[x][1]).with_inputs("question")
                                                 for x in range(len(few_shot))]

In [7]:
treino[0]

Example({'question': 'HIV', 'answer': 'Fever, Chills, Rash, Night sweats (heavy sweating during sleep), Muscle aches, Sore throat, Fatigue, Swollen lymph nodes, Mouth ulcers'}) (input_keys={'question'})

In [8]:
gpt35 = dspy.OpenAI(model="gpt-3.5-turbo-0125", model_type="chat", max_tokens=400, temperature=.4)
dspy.settings.configure(lm=gpt35)

In [9]:
class CoT(dspy.Module):
    def __init__(self):
        self.sabatina = dspy.ChainOfThought("question -> answer")
    
    def forward(self, question):
        return self.sabatina(question=question)

In [10]:
config = dict(max_bootstrapped_demos=5, max_labeled_demos=5)
otimizador = BootstrapFewShot(**config)

In [11]:
modelo = otimizador.compile(CoT(), trainset=treino)

 50%|█████     | 5/10 [00:00<00:00, 30.16it/s]

Bootstrapped 5 full traces after 6 examples in round 0.


In [12]:
modelo(question='Chikungunya')

Prediction(
    rationale='produce the answer. We know that Chikungunya is a viral disease transmitted to humans by infected mosquitoes. Symptoms typically include fever, joint pain, headache, muscle pain, joint swelling, and rash.',
    answer='Fever, Joint pain, Headache, Muscle pain, Joint swelling, Rash'
)

In [13]:
modelo(question='Aspergillosis')

Prediction(
    rationale='produce the answer. Aspergillosis is a fungal infection caused by the Aspergillus mold, so the symptoms will likely involve respiratory issues and can vary depending on the type of aspergillosis (allergic bronchopulmonary aspergillosis, chronic pulmonary aspergillosis, invasive aspergillosis).',
    answer='Cough, Wheezing, Shortness of breath, Fever, Chest pain, Sinusitis symptoms (nasal congestion, runny nose, headache), Fatigue, Unintentional weight loss, Coughing up blood'
)